In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
from IPython.display import clear_output


In [5]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived') # get survived column
y_eval = dfeval.pop('survived')

print(y_train)

CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']

NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = [] # fed to model to make predictions


for fname in CATEGORICAL_COLUMNS:
    vocab = dftrain[fname].unique() # generate list of UNIQUE vals for column
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(fname, vocab))
    # print(vocab)

for fname in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(fname, dtype=tf.float32))


0      0
1      1
2      1
3      1
4      0
      ..
622    0
623    0
624    1
625    0
626    0
Name: survived, Length: 627, dtype: int64


In [6]:
print(y_train)

0      0
1      1
2      1
3      1
4      0
      ..
622    0
623    0
624    1
625    0
626    0
Name: survived, Length: 627, dtype: int64


INFORMATION ANALYZED:
- Most are in 20s, 30s
- Most are male
- Most are in "third" class
- Females have higher chance of survival

CATEGORICAL Vs. NUMERICAL DATA:
- Text vs. Numbers
- Still need to turn this categorical data into numerical

In [7]:
### INPUT FUNCTION - how the model eats data

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def i():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df)) # create dataset object w/ dataframe, then labeled dataframe.
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs) # split dataset into batches
        return ds
    return i # equate to the FUNCTION itself


train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)
    

MODELLL & Training

In [8]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)
clear_output()

In [9]:
print(result['accuracy'])

0.7386364


In [10]:
res = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[2])
print(y_eval.loc[2])
print(res[2]['probabilities'][0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\benli\AppData\Local\Temp\tmp0i4oiqf2\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                        female
age                          58.0
n_siblings_spouses              0
parch                           0
fare                        26.55
class                       First
deck                            C
embark_town           Southampton
alone                           y
Name: 2, dtype: object
1
0.2369338
